In [3]:
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

In [ ]:
# Load the CSV Data
df_one= pd.read_csv('../data/handlebar/Accelerometer_data_merged_ESP1.csv')  # Load the CSV data using pandas
df_two= pd.read_csv('../data/handlebar/Accelerometer_data_merged_ESP2.csv')  # Load the CSV data using pandas
# Convert the NTP column to a datetime object
df_one['NTP'] = pd.to_datetime(df_one['NTP'])
df_two['NTP'] = pd.to_datetime(df_two['NTP'])



,NTP,Acc-Z,curb_scene
0,2024-07-24 10:26:01.863,8.762772,0.0
1,2024-07-24 10:26:01.867,8.753204,0.0
2,2024-07-24 10:26:01.872,8.762772,0.0
3,2024-07-24 10:26:01.877,8.781937,0.0
4,2024-07-24 10:26:01.884,8.753204,0.0


In [ ]:
# Filter the DataFrame to include only the necessary columns
df_filtered = df[['NTP','Acc-Z','curb_scene']]
df_filtered.head()

In [5]:
def handle_missing_values_length(df_filtered):
    # Handle missing values length
    missing_info = {}  # List to store (start_index, current_count)
    current_count = 0
    start_index = None  # To store the start time of missing values
    for index, row in df_filtered.iterrows():
        if np.isnan(row['Acc-Z']):
            if start_index is None:
                start_index = index
                current_count += 1
            if start_index is not None:
                current_count += 1
        # If the value is not NaN, continue sequence broke
        else:
            if start_index is not None:
                missing_info[start_index] = current_count
                start_index = None
                current_count = 0
    # Handle case where the last segment has missing values
    if current_count > 0:
        missing_info[start_index] = current_count

    for key, value in missing_info.items():
        print(key, value)
        if value > 100:
            print("Start Index:", key, "Count:", value)  
    print("No missing value longer than half a second")
    return missing_info


In [6]:
# Function to fill missing values based on the specified conditions
def fill_missing_values(df):
    for index in range(1, len(df)):
        if pd.isnull(df['Acc-Z'].iloc[index]):
            # If the previous value is not NaN, use that value
            if df['curb_scene'].iloc[index - 1] == df['curb_scene'].iloc[index]:
                df.at[index, 'Acc-Z'] = df['Acc-Z'].iloc[index - 1]
                df.at[index, 'NTP'] = df['NTP'].iloc[index - 1] + pd.Timedelta(milliseconds=1)
            else:
                # Find the next non-NaN value below
                for j in range(index + 1, len(df)):
                    if not pd.isnull(df['Acc-Z'].iloc[j]):
                        df.at[index, 'Acc-Z'] = df['Acc-Z'].iloc[j]
                        df.at[index, 'NTP'] = df['NTP'].iloc[j] - pd.Timedelta(milliseconds=1)
                        break
    return df

In [7]:
#check for missing values longer than 0.5 seconds
#missing_info = handle_missing_values_length(df_filtered)

In [8]:
#fill missing values
fill_missing_values(df_filtered)

,NTP,Acc-Z,curb_scene
0,2024-07-24 10:26:01.863,8.762772,0.0
1,2024-07-24 10:26:01.867,8.753204,0.0
2,2024-07-24 10:26:01.872,8.762772,0.0
3,2024-07-24 10:26:01.877,8.781937,0.0
4,2024-07-24 10:26:01.884,8.753204,0.0
...,...,...,...
549230,2024-07-24 11:11:20.851,8.777145,0.0
549231,2024-07-24 11:11:20.857,8.681381,0.0
549232,2024-07-24 11:11:20.866,8.370132,0.0
549233,2024-07-24 11:11:20.870,8.307877,0.0


In [16]:
#Check if there are any missing values
#missing_info= handle_missing_values_length(df_filtered)
#check some random numbers:
print(df_filtered.loc[50 -5:50 +5])
print(df_filtered.loc[0]) 
# print(df_filtered.loc[41458 -10:41458 +5])

                       NTP     Acc-Z  curb_scene
45 2024-07-24 10:26:02.086  8.767563         0.0
46 2024-07-24 10:26:02.091  8.786728         0.0
47 2024-07-24 10:26:02.096  8.791504         0.0
48 2024-07-24 10:26:02.103  8.786728         0.0
49 2024-07-24 10:26:02.106  8.767563         0.0
50 2024-07-24 10:26:02.111  8.762772         0.0
51 2024-07-24 10:26:02.116  8.762772         0.0
52 2024-07-24 10:26:02.121  8.748413         0.0
53 2024-07-24 10:26:02.125  8.767563         0.0
54 2024-07-24 10:26:02.132  8.729263         0.0
55 2024-07-24 10:26:02.136  8.695740         0.0
NTP           2024-07-24 10:26:01.863000
Acc-Z                           8.762772
curb_scene                           0.0
Name: 0, dtype: object


In [10]:
# segments processed
def segment_acceleration_data_no_overlapping(df):
    processed_segments = []
    grouped = df.groupby('curb_scene')
    for name, group in grouped:
        group = group.sort_values(by='NTP')
        # Split the groupmembers into many segments of 100 samples
        for i in range(0, len(group), 100):
            segment = group.iloc[i:i+100]
            if len(segment) < 100:
                break
            acc_z_values = segment['Acc-Z'].values
            curb_scene_value = segment['curb_scene'].iloc[0]
            data = {'curb_scene': curb_scene_value}
            for j, value in enumerate(acc_z_values):
                data[f'Acc-Z_{j+1}'] = value
            new_df = pd.DataFrame([data])
            processed_segments.append(new_df)
    final_df = pd.concat(processed_segments)
    # Save the final DataFrame to a CSV file
    final_df.to_csv('processed_segments.csv')
    return final_df



In [11]:
#segment_acceleration_data_no_overlapping(df_filtered)

In [12]:
# segments processed
def segment_acceleration_data_overlapping(df,overlap):
    processed_segments = []
    step_size = 100 - overlap
    grouped = df.groupby('curb_scene')
    for name, group in grouped:
        group = group.sort_values(by='NTP')
        # Split the groupmembers into many segments of 100 samples
        for i in range(0, len(group), step_size):
            segment = group.iloc[i:i+100]
            if len(segment) < 100:
                break
            acc_z_values = segment['Acc-Z'].values
            curb_scene_value = segment['curb_scene'].iloc[0]
            data = {'curb_scene': curb_scene_value}
            for j, value in enumerate(acc_z_values):
                data[f'Acc-Z_{j+1}'] = value
            new_df = pd.DataFrame([data])
            processed_segments.append(new_df)
    final_df = pd.concat(processed_segments)
    # Save the final DataFrame to a CSV file
    final_df.to_csv('processed_segments_overlap.csv')
    return final_df

In [13]:
segment_acceleration_data_overlapping(df_filtered,50)

,curb_scene,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,Acc-Z_8,Acc-Z_9,...,Acc-Z_91,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100
0,0.0,8.762772,8.753204,8.762772,8.781937,8.753204,8.748413,8.738831,8.748413,8.743622,...,8.786728,8.801086,8.820236,8.786728,8.777145,8.772354,8.781937,8.801086,8.781937,8.753204
0,0.0,8.762772,8.762772,8.748413,8.767563,8.729263,8.695740,8.724472,8.767563,8.767563,...,8.753204,8.772354,8.786728,8.781937,8.786728,8.777145,8.738831,8.820236,8.738831,8.757996
0,0.0,8.781937,8.796295,8.772354,8.772354,8.772354,8.786728,8.781937,8.801086,8.767563,...,8.777145,8.753204,8.767563,8.762772,8.762772,8.753204,8.748413,8.762772,8.772354,8.767563
0,0.0,8.781937,8.753204,8.801086,8.753204,8.829819,8.753204,8.786728,8.815445,8.762772,...,8.805878,8.796295,8.781937,8.801086,8.810669,8.796295,8.805878,8.791504,8.791504,8.772354
0,0.0,8.757996,8.762772,8.777145,8.767563,8.786728,8.777145,8.753204,8.757996,8.777145,...,8.757996,8.757996,8.772354,8.767563,8.757996,8.743622,8.777145,8.743622,8.786728,8.796295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.0,2.849106,4.156342,11.540054,1.398209,-1.881836,2.207458,12.061981,25.627533,17.559082,...,-3.002335,-3.002335,10.764328,19.019547,19.871872,6.004654,9.461884,15.811310,22.572540,20.719421
0,1.0,0.009583,9.605545,12.990936,20.561401,15.883133,11.774689,13.302185,10.544067,17.769760,...,60.429657,60.429657,25.072083,9.495407,9.495407,-14.068329,-19.512741,-11.065994,2.049438,10.376465
0,1.0,7.910446,0.928955,-0.574615,-0.574615,1.824387,0.258575,2.590530,2.590530,12.804184,...,5.104431,0.574615,0.574615,9.016571,19.579788,11.305420,-5.975922,-5.975922,-22.026657,-15.983688
0,1.0,13.273453,18.440140,15.078690,14.274231,9.117126,11.444290,2.896988,2.896988,-5.607223,...,17.372330,11.903976,2.710236,5.200211,11.745956,5.736511,5.736511,10.702087,2.834732,2.834732
